In [1]:
#판다스 모듈 임포트
import pandas as pd

In [2]:
#csv 리딩 용량 확대
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
#n-gram만 붙은 채권리포트 csv 읽기
df_bond = pd.read_csv('bondReport_ngram.csv', encoding='utf-8')

In [15]:
#콜금리 읽기
df_call=pd.read_excel('call_yes.xlsx', encoding='utf-8')

In [16]:
df_call

,date,price,up_down
0,2005-01-01,3.29,상승
1,2005-01-02,3.29,상승
2,2005-01-03,3.27,상승
3,2005-01-04,3.27,상승
4,2005-01-05,3.28,상승
5,2005-01-06,3.21,하락
6,2005-01-07,3.22,상승
7,2005-01-08,3.22,하락
8,2005-01-09,3.22,하락
9,2005-01-10,3.27,하락


In [5]:
#콜금리의 'date' 칼럼 값들 속성 변경. 채권리포트와 병합하기 위함
df_call['date']=pd.to_datetime(df_call['date'])
# df_call=df_call.set_index('date')

In [6]:
#콜금리 내림차순. 채권리포트와 순서 맞추기 위함
df_call=df_call.sort_values(['date'], ascending=[False])
df_call=df_call.reset_index(drop=True)

In [7]:
#채권리포트에서 날짜와 n-gram 칼럼만 추출
df_bngram=df_bond.loc[:,['날짜','n-gram']]

In [8]:
#채권리포트의 '날짜' 칼럼 값들 속성 변경. 콜금리와 병합하기 위함
df_bngram['날짜']=pd.to_datetime(df_bngram['날짜'])
df_bngram.rename(columns={'날짜':'date'},inplace=True) #'날짜' 칼럼 명을 콜금리와 통일

In [9]:
#콜금리와 병합
df_bngram=df_bngram.merge(df_call, on='date',how='left')

In [10]:
#UpDown이 0인 값과 n-gram이 nan인 값 제거
idx = 0
while True:
    try:
        if df_bngram.loc[idx]['up_down'] == 0:
            df_bngram=df_bngram.drop([idx])
        idx+=1
    except:
        break
        
df_bngram=df_bngram.dropna()

In [14]:
df_bngram[df_bngram['up_down']==0]

,date,n-gram,price,up_down


In [ ]:
#인덱스 초기화 및 불필요한 칼럼 제거('price')
df_bngram = df_bngram.reset_index(drop=True)
df_bngram = df_bngram.drop('price',axis=1)

In [ ]:
#데이터 프레임 확인
df_bngram

In [ ]:
#csv 쓰기
df_bngram.to_csv('bond_Labeling.csv',  encoding='utf-8', index=False)